# Import packages

In [ ]:
import google.generativeai as genai
from typing import List, Tuple
import gradio as gr

# Set Gemini API Key

In [ ]:
# TODO: Input API KEY
GOOGLE_API_KEY = ""
genai.configure(api_key=GOOGLE_API_KEY)
model = genai.GenerativeModel("gemini-pro")

# Test if it works
try:
    model.generate_content("test")
    print("Set Gemini API successfully!!")
except:
    print("There seem to be something wrong with API...")

# Task 1: Summarization

In [ ]:
# TODO: Input prompt
prompt_for_summarization = "Please summarize the following article and translate the summarization to chinese for me."

# Function to clear conversation
def reset() -> List:
    """
    Reset the conversation
    """
    return []

# Function to call the model to generate content
def summarization(prompt: str, article: str, temp: float = 1.0) -> List[Tuple[str, str]]:
    '''
      * Parameters

        - prompt: the prompt that we use in this section

        - article: the article to be summarized

        - temp: the temperature parameter of this model. Temperature is used to control the output of the chatbot.
                The higher the temperature is, the more creative response you will get.
    '''
    input = f"{prompt}\n{article}"
    response = model.generate_content(
        input,
        generation_config=genai.types.GenerationConfig(temperature=temp),
        safety_settings=[
          {"category": "HARM_CATEGORY_HARASSMENT","threshold": "BLOCK_NONE",},
          {"category": "HARM_CATEGORY_HATE_SPEECH","threshold": "BLOCK_NONE",},
          {"category": "HARM_CATEGORY_SEXUALLY_EXPLICIT","threshold": "BLOCK_NONE",},
          {"category": "HARM_CATEGORY_DANGEROUS_CONTENT","threshold": "BLOCK_NONE",},
          ]
    )

    return [(input, response.text)]

In [ ]:
# This block construct Gradio UI interface
with gr.Blocks() as demo:
    gr.Markdown(
        """
        # Summarize any article you like !
        """
    )
    with gr.Row():
        with gr.Column():
            prompt_textbox = gr.Textbox(label="Prompt", interactive=False, value=prompt_for_summarization)
            article_textbox = gr.Textbox(label="Article", interactive = True, value = "三隻小豬長大了，決定要離開家裡獨立生活，不過，為了不要被大野狼吃掉，他們決定自己蓋房子。豬大哥說：「我用稻草蓋房子，一天就蓋好了。」豬二哥說：「我用木頭蓋房子，三天就蓋好了。」豬小弟說：「雖然要花十天，我還是要用磚頭蓋房子。」\
    這一天，大野狼來到稻草屋前大叫：「小豬，快讓我進去。」豬大哥說：「我不要，你會把我吃掉。」大野狼說：「那我就把你的房子吹倒，呼～呼～」，一下子豬大哥的稻草屋就被吹倒，豬大哥趕緊跑到豬二哥家裡。接著，大野狼來到小木屋前大叫：「快開門讓我進去。」兩隻小豬說：「我們才不要開門，你會吃掉我們。」大野狼說：「好，看我用力吹倒你的房子，呼～呼～呼～」豬二哥的小木屋也被吹倒，兩隻小豬趕緊跑到豬小弟家裡。\
    最後，大野狼來到磚頭屋前大叫：「快讓我進去。」三隻小豬說：「不開，不開，我們不要變成你的晚餐。」大野狼說：「等我吹倒你的房子，我就把你們吃掉，呼～呼～呼～呼～」大野狼怎麼吹也吹不倒磚頭屋，最後還因為太用力，腳一滑跌到山下，屋裡的三隻小豬高興地在屋子裡唱歌跳舞，豬小弟說：「哥哥，有了堅固的磚頭屋，我們再也不用怕大野狼了。」")
            sent_button = gr.Button(value="Send")
            reset_button = gr.Button(value="Reset")
            temperature_slider = gr.Slider(label="Temperature", minimum=0.0, maximum=1.0, step=0.1, value=0.5)
        with gr.Column():
            chatbot = gr.Chatbot()
        
        sent_button.click(summarization, inputs=[prompt_textbox, article_textbox, temperature_slider], outputs=[chatbot])
        reset_button.click(reset, outputs=[chatbot])

demo.launch(debug=True, share=True)

# Task 2: Role play

In [ ]:
# TODO: Fill in the below two lines: character_for_chatbot and prompt_for_roleplay
# The first one is the character you want your chatbot to play
# The second one is the prompt to make the chatbot be a certain character
character_for_chatbot = "pirate"
prompt_for_roleplay = "Please act as a pirate and play role play game with me."

def reset() -> List[Tuple[str, str]]:
    return roleplay([], prompt_for_roleplay)

def roleplay(chatbot: List[Tuple[str, str]], user_input: str, temp: float=1.0) -> List[Tuple[str, str]]:
    """
    * Parameters
        - chatbot: chat history
        - user_input: the user input of each round of conversation
        - temp: the temperature parameter of this model. Temperature is used to control the output of the chatbot.
              The higher the temperature is, the more creative response you will get.
    """
    try:
        messages = []
        for input_text, response_text in chatbot:
            messages.append({"role": "user", "parts": [input_text]})
            messages.append({"role": "model", "parts": [response_text]})
        
        messages.append({"role": "user", "parts": [user_input]})

        response = model.generate_content(
          messages,
          generation_config=genai.types.GenerationConfig(temperature=temp),
          safety_settings=[
          {"category": "HARM_CATEGORY_HARASSMENT","threshold": "BLOCK_NONE",},
          {"category": "HARM_CATEGORY_HATE_SPEECH","threshold": "BLOCK_NONE",},
          {"category": "HARM_CATEGORY_SEXUALLY_EXPLICIT","threshold": "BLOCK_NONE",},
          {"category": "HARM_CATEGORY_DANGEROUS_CONTENT","threshold": "BLOCK_NONE",},
          ]
        )
        chatbot.append((user_input, response.text))
        
    except Exception as e:
        print(f"Error occured: {e}")
        chatbot.append((input_text, f"Sorry, an error occured: {e}"))

    return chatbot


In [ ]:
first_dialogue = roleplay([], prompt_for_roleplay)

# This block constructs the Gradio UI interface
with gr.Blocks() as demo:
    gr.Markdown("# Task2: The role play\n The chatbot wants to play a role game with you, try interacting with it !")
    with gr.Row():
        with gr.Column():
            description_textbox = gr.Textbox(label="The character that the chatbot play", interactive=True, value=character_for_chatbot)
            input_textbox = gr.Textbox(label="Input", interactive=True, value="")
            gr.Markdown("#  Temperature\n Temperature is used to control the output of the chatbot. The higher the temperature is, the more creative response you will get.")
            temperature_slider = gr.Slider(0.0, 1.0, 0.7, step = 0.1, label="Temperature")
            send_button = gr.Button(value="Send")
        with gr.Column():
            chatbot = gr.Chatbot(value=first_dialogue)
            reset_button = gr.Button(value="Reset")

        send_button.click(roleplay, inputs=[chatbot, input_textbox, temperature_slider], outputs=[chatbot])
        reset_button.click(reset, outputs=[chatbot])

demo.launch(debug = True, share=False) # Turn share=false into true to create a public link

# Task 3: Customize own chatgpt

In [ ]:
def reset() -> List:
    return []
def chat(chatbot: List[Tuple[str, str] | None], input: str, temp: float) -> List[Tuple[str, str]]:
    """
    * Parameters
        - chatbot: Chat history
        - input: The user input of each round of conversation
        - temp: The temperature parameter of this model. Temperature is used to control the output of the chatbot
    """
    chat_history = []
    for user_input, model_response in chatbot:
        chat_history.append({"role": "user", "parts": [user_input]})
        chat_history.append({"role": "model", "parts": [model_response]})
    chat_history.append({"role": "user", "parts": [input]})

    response = model.generate_content(
        contents=chat_history,
        generation_config=genai.types.GenerationConfig(temperature=temp),
        safety_settings=[
            {"category": "HARM_CATEGORY_HARASSMENT","threshold": "BLOCK_NONE",},
            {"category": "HARM_CATEGORY_HATE_SPEECH","threshold": "BLOCK_NONE",},
            {"category": "HARM_CATEGORY_SEXUALLY_EXPLICIT","threshold": "BLOCK_NONE",},
            {"category": "HARM_CATEGORY_DANGEROUS_CONTENT","threshold": "BLOCK_NONE",},
        ]
    )
    chatbot.append((input, response.text))

    return chatbot

In [ ]:
with gr.Blocks() as demo:
    gr.Markdown("# Chat with Chatbot!!")
    with gr.Row():
        with gr.Column():
            input_textbox = gr.Textbox(label="Input text", interactive=True)
            gr.Markdown("# Temperature \n Temperature is used to control the output of chatbot. The higher the temperature is, the more creative output you will get. ")
            temp_slider= gr.Slider(label="Temperature", minimum=0.0, maximum=1.0, step=0.1, value=0.7)
            send_button = gr.Button(value="Send")
            reset_button = gr.Button(value="Reset")
        with gr.Column():
            chatbot = gr.Chatbot()
        send_button.click(chat, inputs=[chatbot, input_textbox, temp_slider], outputs=[chatbot])
        reset_button.click(reset, outputs=[chatbot])

demo.launch(debug=True, share=True)